In [1]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import os
import numpy as np

In [2]:
input_saved_model_dir = './Exported_model_V1/saved_model/'
output_saved_model_dir = './Exported_model_V1/converted_model/'

In [9]:
# the info about input size was taken from pipeline.config file

# Create some dummy data which we use for benchmarking and conversion
data = tf.constant(np.empty([1, 640, 640, 3], np.uint8))

## Benchmark original model with dummy data

In [29]:
model_original = tf.saved_model.load(input_saved_model_dir)

In [31]:
%%timeit
for i in range(100):
    model_original(data)

2.98 s ± 19.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Convert model to TensorRT

In [3]:
params = tf.experimental.tensorrt.ConversionParams(
    precision_mode='FP16')
converter = trt.TrtGraphConverterV2(input_saved_model_dir=input_saved_model_dir, conversion_params=params)

INFO:tensorflow:Linked TensorRT version: (6, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (6, 0, 1)


In [4]:
converter.convert()

In [5]:
def my_input_fn():
    yield [data]

In [6]:
converter.build(input_fn=my_input_fn)

In [7]:
converter.save(output_saved_model_dir)

INFO:tensorflow:Assets written to: ./Exported_model_V1/converted_model/assets


## Benchmark converted model

In [8]:
model_converted = tf.saved_model.load(output_saved_model_dir)

In [10]:
model = model_converted.signatures["serving_default"]

In [11]:
%%timeit
for i in range(100):
    model(data)

1.72 s ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Conclusion: tensor RT gives about 2x speed up wrt original model before conversion